In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties

from utils.models import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

dataA2 = pd.read_csv("../Version3/Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("../Version3/Data/A1.csv")

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [2]:
car_0 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_0.csv"), encoding='utf-8')
car_1 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_1.csv"), encoding='utf-8')
car_2 = pd.read_csv(os.path.join("../Version3/Data/CarData/full_2.csv"), encoding='utf-8')
car_out = pd.read_csv(os.path.join("../Version3/Data/CarData/full_out.csv"), encoding='utf-8')
car_overlap = pd.read_csv(os.path.join("../Version3/Data/CarData/overlap_data.csv"), encoding='utf-8')

motor_0 = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_0.csv"), encoding='utf-8')
motor_1 = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_1.csv"), encoding='utf-8')
motor_out = pd.read_csv(os.path.join("../Version3/Data/MotorData/full_out.csv"), encoding='utf-8')
motor_overlap = pd.read_csv(os.path.join("../Version3/Data/MotorData/overlap_data.csv"), encoding='utf-8')

pass_0 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_0.csv"), encoding='utf-8')
pass_1 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_1.csv"), encoding='utf-8')
pass_out = pd.read_csv(os.path.join("../Version3/Data/PassData/full_out.csv"), encoding='utf-8')
pass_overlap = pd.read_csv(os.path.join("../Version3/Data/PassData/overlap_data.csv"), encoding='utf-8')

### 這是由拓樸得出來的特徵，可新增於拓樸訓練，但不增加在full_data

In [3]:

car_out['type'] = 'out'
car_overlap['type'] = 'overlap'
motor_out['type'] = 'out'
motor_overlap['type'] = 'overlap'
pass_out['type'] = 'out'
pass_overlap['type'] = 'overlap'

car_out_overlap = pd.concat([car_out, car_overlap])
motor_out_overlap = pd.concat([motor_out, motor_overlap])
pass_out_overlap = pd.concat([pass_out, pass_overlap])

# 使用concat較為合理，因為Mapper個別處理過資料，這樣比較公平
full_data = pd.concat([car_0, car_1, car_2, car_out_overlap, motor_0, motor_1, motor_out_overlap, pass_0, pass_1, pass_out_overlap])

### 行人資料需要特別處理，因為他們沒有以下特徵

In [4]:
pass_0['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_0['當事者區分-類別-大類別名稱-車種'] = '人'
pass_1['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_1['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out['當事者區分-類別-大類別名稱-車種'] = '人'
pass_overlap['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_overlap['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out_overlap['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out_overlap['當事者區分-類別-大類別名稱-車種'] = '人'

In [5]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    
    # 路面狀況
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    # 車道
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # 當事人
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    '道路型態大類別名稱', '事故位置大類別名稱',
    '車道劃分設施-分向設施大類別名稱',
    '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    '車輛撞擊部位大類別名稱-其他',
    '肇因研判大類別名稱-主要',

    # 子類別
    '道路型態子類別名稱', '事故位置子類別名稱', '事故類型及型態子類別名稱', '肇因研判子類別名稱-主要',
    '當事者區分-類別-子類別名稱-車種', '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初',
    '車輛撞擊部位子類別名稱-其他', '肇因研判子類別名稱-個別',
    
    '死亡'
]

# select data
car_0 = car_0[select_lst]
car_1 = car_1[select_lst]
car_2 = car_2[select_lst]
motor_0 = motor_0[select_lst]
motor_1 = motor_1[select_lst]
pass_0 = pass_0[select_lst]
pass_1 = pass_1[select_lst]

full_data = full_data[select_lst]

select_lst.append('type')
motor_out_overlap = motor_out_overlap[select_lst]
car_out_overlap = car_out_overlap[select_lst]
pass_out_overlap = pass_out_overlap[select_lst]

## 唯一值檢查
第一區塊是完整模型的資料<br/>
第二區塊是多個模型個別進行模型，所以這裡需要量化後自動drop

In [6]:
print('詳細群體')
for data in [car_0, car_1, car_2, car_out_overlap, motor_0, motor_1, motor_out_overlap, pass_0, pass_1, pass_out_overlap, full_data]:
    columns_to_drop = []
    for column in data.columns:
        if data[column].nunique() == 1:
            columns_to_drop.append(column)
    print(columns_to_drop)
    data.drop(columns=columns_to_drop, inplace=True)

詳細群體
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
[]
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
[]
['路面狀況-路面缺陷名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱', '號誌-號誌動作名稱', '行動電話或電腦或其他相類功能裝置名稱', '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種']
['道路障礙-視距品質名稱', '道路障礙-視距名稱', '行動電話或電腦或其他相類功能裝置名稱', '當事者區分-類別-大類別名稱-車種']
['行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃', '當事者區分-類別-大類別名稱-車種']
[]


In [7]:
from sklearn.ensemble import RandomForestClassifier

def drop_low_importance_features(data, threshold=0.01):
    # One-hot 編碼資料
    X = pd.get_dummies(data.drop(columns=['死亡']))
    y = data['死亡']

    # 訓練 RandomForest 模型
    model = RandomForestClassifier(random_state=42)
    model.fit(X, y)

    # 計算特徵重要性
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

    # 篩選重要性高於 threshold 的 dummy 特徵
    selected_dummy_features = feature_importances[feature_importances > threshold].index.tolist()

    # 找回原始的類別特徵名稱
    original_features = data.drop(columns=['死亡']).columns
    selected_features = set()

    for feature in original_features:
        # 如果原始特徵的任何 dummy 特徵被選中，就保留原始特徵
        if any(dummy.startswith(f"{feature}_") for dummy in selected_dummy_features) or feature in selected_dummy_features:
            selected_features.add(feature)

    # 回到原始資料集中的選擇特徵
    reduced_data = data[list(selected_features) + ['死亡']]
    print(f"原始特徵數: {data.shape[1]}, 保留特徵數: {len(selected_features)}")
    return reduced_data

car0_reduced = drop_low_importance_features(car_0)
car1_reduced = drop_low_importance_features(car_1)
car2_reduced = drop_low_importance_features(car_2)
car_out_overlap_reduced = drop_low_importance_features(car_out_overlap)
motor0_reduced = drop_low_importance_features(motor_0)
motor1_reduced = drop_low_importance_features(motor_1)
motor_out_overlap_reduced = drop_low_importance_features(motor_out_overlap)
pass0_reduced = drop_low_importance_features(pass_0)
pass1_reduced = drop_low_importance_features(pass_1)
pass_out_overlap_reduced = drop_low_importance_features(pass_out_overlap)
full_data_reduced = drop_low_importance_features(full_data)

原始特徵數: 35, 保留特徵數: 12
原始特徵數: 35, 保留特徵數: 17
原始特徵數: 35, 保留特徵數: 14
原始特徵數: 38, 保留特徵數: 8
原始特徵數: 35, 保留特徵數: 14
原始特徵數: 35, 保留特徵數: 16
原始特徵數: 38, 保留特徵數: 17
原始特徵數: 30, 保留特徵數: 18
原始特徵數: 33, 保留特徵數: 17
原始特徵數: 35, 保留特徵數: 18
原始特徵數: 37, 保留特徵數: 9


In [8]:
car_0_dummy = pd.get_dummies(car0_reduced)
car_1_dummy = pd.get_dummies(car1_reduced)
car_2_dummy = pd.get_dummies(car2_reduced)
car_out_overlap_dummy = pd.get_dummies(car_out_overlap_reduced)
motor_0_dummy = pd.get_dummies(motor0_reduced)
motor_1_dummy = pd.get_dummies(motor1_reduced)
motor_out_overlap_dummy = pd.get_dummies(motor_out_overlap_reduced)
pass_0_dummy = pd.get_dummies(pass0_reduced)
pass_1_dummy = pd.get_dummies(pass1_reduced)
pass_out_overlap = pd.get_dummies(pass_out_overlap_reduced)

full_data_dummy = pd.get_dummies(full_data_reduced)

In [9]:
car_0_X, car_0_y = get_train_test_data(car_0_dummy)
car_1_X, car_1_y = get_train_test_data(car_1_dummy)
car_2_X, car_2_y = get_train_test_data(car_2_dummy)
car_out_overlap_X, car_out_overlap_y = get_train_test_data(car_out_overlap_dummy)
motor_0_X, motor_0_y = get_train_test_data(motor_0_dummy)
motor_1_X, motor_1_y = get_train_test_data(motor_1_dummy)
motor_out_overlap_X, motor_out_overlap_y = get_train_test_data(motor_out_overlap_dummy)
pass_0_X, pass_0_y = get_train_test_data(pass_0_dummy)
pass_1_X, pass_1_y = get_train_test_data(pass_1_dummy)
pass_out_overlap_X, pass_out_overlap_y = get_train_test_data(pass_out_overlap)

full_data_X, full_data_y = get_train_test_data(full_data_dummy)

In [10]:
print(car_0_y.value_counts())
print(car_1_y.value_counts())
print(car_2_y.value_counts())
print(car_out_overlap_y.value_counts())
print(motor_0_y.value_counts())
print(motor_1_y.value_counts())
print(motor_out_overlap_y.value_counts())
print(pass_0_y.value_counts())
print(pass_1_y.value_counts())
print(pass_out_overlap_y.value_counts())

print(full_data_y.value_counts())

0    57472
1      285
Name: y, dtype: int64
0    43045
1      175
Name: y, dtype: int64
0    2084
1      35
Name: y, dtype: int64
0    8772
1      88
Name: y, dtype: int64
0    69156
1      318
Name: y, dtype: int64
0    32699
1       87
Name: y, dtype: int64
0    5353
1      18
Name: y, dtype: int64
0    529
1     11
Name: y, dtype: int64
0    2373
1      33
Name: y, dtype: int64
0    183
1     14
Name: y, dtype: int64
0    221666
1      1064
Name: y, dtype: int64


In [11]:
print(car_0_X.shape)
print(car_1_X.shape)
print(car_2_X.shape)
print(car_out_overlap_X.shape)
print(motor_0_X.shape)
print(motor_1_X.shape)
print(motor_out_overlap_X.shape)
print(pass_0_X.shape)
print(pass_1_X.shape)
print(pass_out_overlap_X.shape)
print(full_data_X.shape)

assert car_0_X.shape[0] + car_1_X.shape[0] + car_2_X.shape[0] + car_out_overlap_X.shape[0] + \
    motor_0_X.shape[0] + motor_1_X.shape[0] + motor_out_overlap_X.shape[0] + \
        pass_0_X.shape[0] + pass_1_X.shape[0] + pass_out_overlap_X.shape[0] == full_data_X.shape[0]

(57757, 100)
(43220, 164)
(2119, 223)
(8860, 55)
(69474, 117)
(32786, 114)
(5371, 251)
(540, 93)
(2406, 117)
(197, 117)
(222730, 64)


## 全部大類別資料kfold計算到motor0開始

In [29]:
import pickle
import time
import gc

models = [
    # ("pass_0", pass_0_X, pass_0_y), 完成
    # ("pass_1", pass_1_X, pass_1_y), 完成
    # ("pass_out_overlap", pass_out_overlap_X, pass_out_overlap_y), 完成
    ("car_0", car_0_X, car_0_y),
    ("car_1", car_1_X, car_1_y),
    # ("car_2", car_2_X, car_2_y), 完成
    # ("car_out_overlap", car_out_overlap_X, car_out_overlap_y), 完成
    ("motor_0", motor_0_X, motor_0_y),
    ("motor_1", motor_1_X, motor_1_y),
    # ("motor_out_overlap", motor_out_overlap_X, motor_out_overlap_y), 完成
    ("full_data", full_data_X, full_data_y),
]

for seed in range(40, 50):
    # XGBoost no fold
    for name, X, y in models:
        print(f'{name} xgboost start')
        start_time = time.time()
        y_xgb, decision_scores_xgb, indices_xgb = xgboost_cm_gridsearch(X.astype(float), y, random_state=seed)
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        save_dir = f"../Models/ModelPerformanceSeed/{seed}/xgboost"
        os.makedirs(save_dir, exist_ok=True)
        
        with open(f"{save_dir}/{name}.pkl", "wb") as f:
            pickle.dump({
                'y': y_xgb,
                'decision_scores': decision_scores_xgb,
                'indices': indices_xgb,
                'elapsed_time': elapsed_time
            }, f)
        print(f'{name} xgboost done in {elapsed_time:.2f} seconds')
        del X, y, y_xgb, decision_scores_xgb
        gc.collect()

    # Logistic no fold
    for name, X, y in models:
        print(f'{name} logistic start')
        start_time = time.time()
        y_log, decision_scores_log, indices_log = logistic_cm_gridsearch(X.astype(float), y, random_state=seed)
        end_time = time.time()
        elapsed_time = end_time - start_time

        save_dir = f"../Models/ModelPerformanceSeed/{seed}/logistic"
        os.makedirs(save_dir, exist_ok=True)
        
        with open(f"{save_dir}/{name}.pkl", "wb") as f:
            pickle.dump({
                'y': y_log,
                'decision_scores': decision_scores_log,
                'indices': indices_log,
                'elapsed_time': elapsed_time
            }, f)
        print(f'{name} logistic done in {elapsed_time:.2f} seconds')
        del X, y, y_log, decision_scores_log
        gc.collect()

    # SVC no fold
    for name, X, y in models:
        print(f'{name} svc start')
        start_time = time.time()
        y_svc, decision_scores_svc, indices_svc = linear_svc_cm_gridsearch(X.astype(float), y, random_state=seed)
        end_time = time.time()
        elapsed_time = end_time - start_time

        save_dir = f"../Models/ModelPerformanceSeed/{seed}/svc"
        os.makedirs(save_dir, exist_ok=True)
        
        with open(f"{save_dir}/{name}.pkl", "wb") as f:
            pickle.dump({
                'y': y_svc,
                'decision_scores': decision_scores_svc,
                'indices': indices_svc,
                'elapsed_time': elapsed_time
            }, f)
        print(f'{name} svc done in {elapsed_time:.2f} seconds')
        del X, y, y_svc, decision_scores_svc
        gc.collect()

car_0 xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 400}
car_0 xgboost done in 567.54 seconds
car_1 xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 400}
car_1 xgboost done in 651.12 seconds
motor_0 xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 400}
motor_0 xgboost done in 700.70 seconds
motor_1 xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}
motor_1 xgboost done in 382.06 seconds
full_data xgboost start
Best parameters found by GridSearchCV: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 400}
full_data xgboost done in 1191.87 seconds
car_0 logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalt

c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 100, 'loss': 'hinge'}
car_0 svc done in 137.15 seconds
car_1 svc start


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 100, 'loss': 'hinge'}
car_1 svc done in 188.72 seconds
motor_0 svc start


...

Best parameters found by GridSearchCV: {'C': 100, 'loss': 'hinge'}
car_1 svc done in 195.20 seconds
motor_0 svc start


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 10, 'loss': 'hinge'}
motor_0 svc done in 121.79 seconds
motor_1 svc start


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 100, 'loss': 'hinge'}
motor_1 svc done in 190.16 seconds
full_data svc start
Best parameters found by GridSearchCV: {'C': 0.1, 'loss': 'squared_hinge'}
full_data svc done in 410.72 seconds
